**Exercicio con Nominatim**

In [ ]:
# 0. Crea un dataframe co  nome de 3 lugares da Coruña ou do resto de Galicia que recomendarías para comer, engade un comentario persoal
# 1. Busca a través da API de Nominatim (versión privada) información dos lugares que recomendarías
# 2. Crea un dataframe coa seguinte información: nome, enderezo, cidade, coordenadas e comentario
# 3. Visualiza os 3 lugares nun mapa con folium
# 4. Engade a información das túas recomendacións ao ficheiro colaborativo e fai un PR para compartilas con todos
# 5. Pinta un novo mapa directamente sobre os datos do ficheiro compartido na rede (utiliza clúster de marcas)

# Lembra facer un fork do repo antes de enviar o ficheiro cos teus cambios

# REPO: https://github.com/bigdatawirtz/restaurantes

In [ ]:
# Creo de xeito manual un Dataframe con 3 recomendacións persoais

import pandas as pd

recomendacions = pd.DataFrame(
    { 'nome' : ['Barazal III','O Fiuza','Os Ventis'],
    'comentario' : ['O mellor pincho de tortilla da cidade','O mellor pulpo da Coruña','Tapas baratas e unha terraza macanuda']})
recomendacions

In [ ]:
# Nesta Cell podo probar individualmente as miñas consultas

# Comprobo que a función Search de Nominatim encontra os lugares polos que pregunto e miro que datos devolve

import requests

busqueda = 'O Fiuza'

search_url = 'https://nominatim.openstreetmap.org/search'
#search_url = 'http://localhost:8080/search'
payload = {'q': busqueda ,'format':'json'}

search_response = requests.get(search_url,params=payload)

latitude = search_response.json()[0]['lat']
longitude = search_response.json()[0]['lon']
search_response.json()

In [ ]:
# Automatizo as consultas iterando sobre os datos
# Gardo os resultados e creo un novo dataframe


datos_restaurantes = []

#search_url = 'https://nominatim.openstreetmap.org/search'
#search_url = 'http://localhost:8080/search'
search_url = 'http://34.79.16.254/'

for idx, recomendacion in recomendacions.iterrows():
    search_payload = {'q': recomendacion.nome ,'format':'json'}
    search_response = requests.get(search_url,params=search_payload)
    latitude = search_response.json()[0]['lat']
    longitude = search_response.json()[0]['lon']
    osm_id = search_response.json()[0]['osm_id']

    datos_restaurantes.append([recomendacion.nome,latitude,longitude])

restaurantes = pd.DataFrame(datos_restaurantes,columns=['nome','latitude','lonxitude'])
restaurantes


In [ ]:
# Cruzo os dous dataframes para ter todos os datos xuntos
restaurantes = restaurantes.merge(recomendacions)
restaurantes

In [ ]:
# Exporto os datos a un ficheiro coa intención de envialo a un repositorio común
restaurantes.to_csv('restaurantes.csv',sep=';',index=None)

In [ ]:
## Aquí viría a parte onde creo un PR con Git para engadir as miñas suxestión a un ficheiro común

In [ ]:
# Represento os datos do meu dataframe sobre o mapa

import folium

m = folium.Map(location=[43,-8],zoom_start=10)
for idx, restaurante in restaurantes.iterrows():
    folium.Marker([restaurante.latitude,restaurante.lonxitude],
    tooltip=restaurante.nome,
    popup=(f'{restaurante.nome} ({restaurante.comentario})'),
    icon=folium.Icon(color='red',icon='beer', prefix='fa')). \
    add_to(m)

sw = restaurantes[['latitude', 'lonxitude']].min().values.tolist()
ne = restaurantes[['latitude', 'lonxitude']].max().values.tolist()

m.fit_bounds([sw, ne])      
m

In [ ]:
# Leo os datos remotos do repositorio, onde hai un ficheiro con todas as colaboracións

url_restaurantes = 'https://raw.githubusercontent.com/bigdatawirtz/restaurantes/main/restaurantes.csv'
restaurantes_total = pd.read_csv(url_restaurantes,sep=';')
restaurantes_total

In [ ]:
# Represento os puntos sobre o mapa
# O único que cambia é o dataframe que utilizo como orixe de datos

import folium

m = folium.Map(location=[43,-8])
for idx, restaurante in restaurantes_total.iterrows():
    folium.Marker([restaurante.latitude,restaurante.lonxitude],
    tooltip=restaurante.nome,
    popup=(f'{restaurante.nome} ({restaurante.comentario})'),
    icon=folium.Icon(color='red',icon='beer', prefix='fa')). \
    add_to(m)

sw = restaurantes_total[['latitude', 'lonxitude']].min().values.tolist()
ne = restaurantes_total[['latitude', 'lonxitude']].max().values.tolist()

m.fit_bounds([sw, ne])      
m

In [ ]:
m.save('mapa_colaborativo.html')